In [316]:
import pandas as pd
import numpy as np
from ipywidgets import interact

In [317]:
def simplify_column_names( df: pd.DataFrame, col_names: list, prefixes: list) -> pd.DataFrame:
    '''
    For a given dataframe, renames lists of column names with a shared prefix and applies a sequential counter after each prefix.

    Example: In dataframe df, columns groups [['hello', 'world'], ['foo','bar']], prefixes ['a','b'] would yield new column names
    a_1, a_2, b_1, b_2
    '''
    for c, p in zip(col_names, prefixes):
        rename_map = { i : f"{p}_{str(c.index(i) + 1)}" for i in c}
        df.rename(rename_map, axis=1, inplace=True )
    
    return df

In [318]:
def strip_chars(string : str) -> int:
    '''
    Strips all non-numeric characters and returns the rest as a single int
    '''
    string = string.replace('%','')
    res = ''.join([i for i in string.split() if i.isdigit()])
    if res:
        return int(''.join([i for i in string.split() if i.isdigit()]))
    else:
        return 0

In [319]:
pd.set_option('display.max_colwidth', None)

In [320]:
data_dir = '../data/'

In [321]:
raw_data = pd.read_csv(data_dir + '2021_rws.csv', encoding='cp1252')

In [322]:
raw_data.shape

(1512, 109)

Drop columns we know we will not be examining

- Columns pertaining to collaborating remotely with onsite employees (column indices 65-92)

In [323]:
remote_onsite_cols = raw_data.columns[65:93]

In [324]:
df = raw_data.drop( columns= remote_onsite_cols)

In [325]:
df.shape

(1512, 81)

Trim leading/trailing whitespace in columns

In [326]:
column_name_trimmed = { k : k.strip() for k in df.columns}
df.rename(column_name_trimmed, axis=1, inplace=True)

In [327]:
cols = list(df.columns)

Renaming columns from original survey questions to more workable column names

In [328]:
# Columns indices from column mapping file
# Values maintained as lists to support simplify_column_names function
column_map = {
    'demographics' : cols[1:4] + cols[7:9],
    'company' : cols[4:7],
    'remote_work_time' : cols[9:14],
    'remote_policy' : cols[14:15] + cols[18:25],
    'split_work_day' : cols[15:18],
    'remote_culture' : cols[25:37],
    'use_of_time_onsite' : cols[37:42],
    'use_of_time_remote' : cols[42:47],
    'compensation_hypothetical' : cols[47:49],
    'barriers' : cols[49:65],
    'remote_personal_behavior' : cols[65:68],
    'remote_management' : cols[68:79] + cols[80:81],
    'productivity' : cols[79:80]
    
}

In [329]:
df = simplify_column_names( df, column_map.values() , column_map.keys() )

In [330]:
df.rename( {'Response ID' : 'id'}, axis=1, inplace=True)

In [331]:
df.columns

Index(['id', 'demographics_1', 'demographics_2', 'demographics_3', 'company_1',
       'company_2', 'company_3', 'demographics_4', 'demographics_5',
       'remote_work_time_1', 'remote_work_time_2', 'remote_work_time_3',
       'remote_work_time_4', 'remote_work_time_5', 'remote_policy_1',
       'split_work_day_1', 'split_work_day_2', 'split_work_day_3',
       'remote_policy_2', 'remote_policy_3', 'remote_policy_4',
       'remote_policy_5', 'remote_policy_6', 'remote_policy_7',
       'remote_policy_8', 'remote_culture_1', 'remote_culture_2',
       'remote_culture_3', 'remote_culture_4', 'remote_culture_5',
       'remote_culture_6', 'remote_culture_7', 'remote_culture_8',
       'remote_culture_9', 'remote_culture_10', 'remote_culture_11',
       'remote_culture_12', 'use_of_time_onsite_1', 'use_of_time_onsite_2',
       'use_of_time_onsite_3', 'use_of_time_onsite_4', 'use_of_time_onsite_5',
       'use_of_time_remote_1', 'use_of_time_remote_2', 'use_of_time_remote_3',
       'us

In [332]:
df['id'].unique()

array([   1,    2,    3, ..., 1510, 1511, 1512], dtype=int64)

In [333]:
@interact

def df_answer_explorer(column = df.columns):
    '''
    Allows quick views of all unique answers for a given response question
    
    '''
    if column != 'id': 
        # answers = list(df[column].unique())
        column_name_parts = column.rsplit('_' ,1 )
        response_question = column_map[column_name_parts[0]][int(column_name_parts[1]) - 1]
        vk_column_counts = { df[df[column] == i].shape[0] : i for i in df[column].unique() }
        sorted_kv_column_counts = { vk_column_counts[i] : i for i in sorted(vk_column_counts, reverse=True)}
        
        print("Survey question:")
        print(response_question)
        print("")
        result_df = pd.DataFrame(sorted_kv_column_counts.values(), index=sorted_kv_column_counts.keys())
        result_df.rename({0 : 'Count'}, axis=1, inplace=True)
        return result_df
    else:
        print(f"'id' is a numerical range from 1 to {df.shape[0]}")

interactive(children=(Dropdown(description='column', options=('id', 'demographics_1', 'demographics_2', 'demog…

## Reforming survey responses to numeric values where possible

In [334]:
tidy_df = df.copy()

### Binarize responses across all columns where possible
'Yes' = 1; 'No' = 0

In [366]:
for col in tidy_df.columns:
    if tidy_df[col].dtype == 'O':
        responses = tidy_df[col].str.lower().unique()
        if len(responses) == 2 and 'yes' in responses and 'no' in responses:
            tidy_df[col] = tidy_df[col].str.lower()
            tidy_df[col] = np.where(tidy_df[col] == 'yes', 1, 0)

### Recreate "year born" as "age" based on year this survey took place (2021)

In [335]:
tidy_df['demographics_1'] = 2021 - df['demographics_1']

### Recreate "Metro or Regional" as "Metro" and make binary

In [336]:
tidy_df['demographics_5'] = np.where(tidy_df['demographics_5'] == 'Metro', 1, 0)

### "Remote work time" percentage questions as numeric

**Note**: In these questions, "Less than 10%"" is distinct from "10%" and "Rarely or never", but still represents that the responder went onsite at least for a few hours in the week. The result of the function would combine it with those distinctly answering "10%", so I have deliberately created a new distinct value splitting the difference - 5%

In [337]:
for i in range(1,6):
    tidy_df[f"remote_work_time_{i}"] = np.where(tidy_df[f"remote_work_time_{i}"] == 'Less than 10% of my time', '5', tidy_df[f"remote_work_time_{i}"])
    tidy_df[f"remote_work_time_{i}"] = tidy_df[f"remote_work_time_{i}"].apply(strip_chars)

### Remote policy

In [338]:
@interact

def tidy_df_answer_explorer(column = tidy_df.columns):
    '''
    Allows quick views of all unique answers for a given response question
    
    '''
    if column != 'id': 
        # answers = list(df[column].unique())
        column_name_parts = column.rsplit('_' ,1 )
        response_question = column_map[column_name_parts[0]][int(column_name_parts[1]) - 1]
        vk_column_counts = { tidy_df[tidy_df[column] == i].shape[0] : i for i in tidy_df[column].unique() }
        sorted_kv_column_counts = { vk_column_counts[i] : i for i in sorted(vk_column_counts, reverse=True)}
        
        print("Survey question:")
        print(response_question)
        print("")
        result_df = pd.DataFrame(sorted_kv_column_counts.values(), index=sorted_kv_column_counts.keys())
        result_df.rename({0 : 'Count'}, axis=1, inplace=True)
        return result_df
    else:
        print(f"'id' is a numerical range from 1 to {df.shape[0]}")

interactive(children=(Dropdown(description='column', options=('id', 'demographics_1', 'demographics_2', 'demog…